In [41]:
# import libraries
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# api requests
# Currently we are on congress 117


# Senator api request
Senator_response = requests.get(
    'https://api.propublica.org/congress/v1/117/senate/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)
# House api request
House_response = requests.get(
    'https://api.propublica.org/congress/v1/117/house/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)

#convert responses into json
senate_json_response = Senator_response.json()
house_json_response = House_response.json()

In [42]:

# Senator array
Senate = senate_json_response['results'][0]["members"]
print(len(Senate))

# House array
House = house_json_response['results'][0]["members"]    
print(len(House))

102
440


In [43]:
#Congress Array
Congress = Senate + House
print(len(Congress))

542


In [44]:
# Convert array in Dataframe
df = pd.DataFrame(Congress)

In [45]:
#dropping unnessary columns from dataframe
df = df.drop(['date_of_birth', 'twitter_account','facebook_account','youtube_account', 'rss_url','contact_form', 'in_office', 'ideal_point', 'contact_form','next_election',
'total_votes', 'missed_votes', 'total_present', 'last_updated', 'office', 'phone', 'fax', 'at_large', 'geoid', 'cook_pvi','dw_nominate', 'votes_with_party_pct', 'votes_with_party_pct', 'missed_votes_pct','votes_against_party_pct'], axis=1)

In [46]:
df.keys()

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'gender', 'party', 'leadership_role',
       'govtrack_id', 'cspan_id', 'votesmart_id', 'icpsr_id', 'crp_id',
       'google_entity_id', 'fec_candidate_id', 'url', 'seniority', 'ocd_id',
       'state', 'senate_class', 'state_rank', 'lis_id', 'district'],
      dtype='object')

In [47]:
# Column that we are adding to the dataframe
df['image'] = "None"
# VTO20E = Voted to overturn 2020 election
df['VTO20E'] = False
# VTITAI = Voted to impeach Trump after inserection
df['VTITAI'] = True    

In [49]:
# States for scraping
states = ["alabama", 'alaska',"arizona", "arkansas", "california", "colorado",'connecticut','delaware', 'florida','georgia', 'hawaii','idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri','montana', 'nebraska','nevada', 'new%20hampshire', 'new%20jersey','new%20mexico', 'new%20york', 'north%20carolina', 'north%20dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania','puerto%20rico', 'rhode%20island', 'south%20carolina', 'south%20dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west%20virginia', 'wisconsin', 'wyoming' ]

Main_url = 'https://www.congress.gov/search?searchResultViewType=expanded&q={%22congress%22:[%22116%22],%22source%22:[%22members%22],%22search%22:%22'
back_of_Main_url = "%22}"
top_images_url = 'https://congress.gov'
#reps with no photos on Congress.gov
no_photos = []
Congress_dic = {}
Congress_dic['congress_info'] = []

In [50]:
# Checking to see if there is a picture on the website
def CheckForImg(str):
    try:
        rep_Img = rep.find("img")['src']
        return True
    except:
        return False

In [51]:
# State Abbrevation Map Key
state_abrv = {"alabama":'AL', 'alaska':'AL',"arizona":'AZ', "arkansas":'AK', "california":'CA', "colorado":'CO','connecticut':'CT','delaware':'DE', 'florida':'FL','georgia':'GA', 'hawaii':'HI','idaho':'ID', 'illinois':'IL', 'indiana':'ID', 'iowa':'IA', 'kansas':'KS', 'kentucky':'KY', 'louisiana':'LA', 'maine':'ME', 'maryland':'MD', 'massachusetts':'MA', 'michigan':"MI", 'minnesota':'MN', 'mississippi':'MS', 'missouri':'MS','montana':'MT', 'nebraska':'NE','nevada':'NV', 'new%20hampshire':'NH', 'new%20jersey':'NJ','new%20mexico':'NM', 'new%20york':'NY', 'north%20carolina':'NC', 'north%20dakota':'ND', 'ohio':'OH', 'oklahoma':'OK', 'oregon':'OR', 'pennsylvania':'PA','puerto%20rico':'PR','rhode%20island':'RI', 'south%20carolina':'SC', 'south%20dakota':'SD', 'tennessee':'TN', 'texas':'TX', 'utah':'UT', 'vermont':'VT', 'virginia':'VA', 'washington':'WA', 'west%20virginia':'WV', 'wisconsin':'WI', 'wyoming':'WY'}

In [52]:
# # for loop that is going thru every state
for state in states: 
    state_res = requests.get(Main_url+state+back_of_Main_url)
    state_soup = BeautifulSoup(state_res.text, 'html.parser')
    Rep_results = state_soup.findAll("li", {"class":"expanded"})

    for rep in Rep_results:
        photo = CheckForImg(rep)
        rep_name = rep.find("span", {"class":"result-heading"}).text.split(' ')[1:]
        rep_index = df.loc[(df['state'] == state_abrv[state]) & (df['last_name'] == rep_name[0][:-1])].index
        if photo:
            df.at[rep_index, 'image'] = rep_Img = rep.find("img")['src']

In [95]:
# Switching all House Rep. that they didnt want to impeach trump
df.loc[(df.party == "R"), ['VTITAI']] = 0.0

In [96]:
# House Members who didnt appear to vote on impeachmenting charges
no_vote = ["Kay Granger", "Andy Harris", "Greg Murphy", "Daniel Webster"]
for pussy in no_vote:
    name = pussy.split(" ")
    rep_index = df.loc[(df['party'] == "R") & (df['last_name'] == name[1]) & (df['first_name'] == name[0])].index
    df.at[rep_index, 'VTITAI'] = None

In [104]:
# Republican House & Senate members who voted to impeach Trump
RepHouseHeros = ["Liz, Cheney", "Anthony, Gonzalez", "Jaime, Herrera Beutler", "John, Katko", "Adam, Kininger", "Peter, Meijer", "Dan, Newhouse", "Tom, Rice", "Fred, Upton", "David, Valadao", "Susan, Collins", "Lisa, Murkowski", "Mitt, Romney", "Ben, Sasse", " Bill, Cassidy", "Patrick, Toomey", "Richard, Burr"]

for hero in RepHouseHeros:
    name = hero.split(", ")
    rep_index = df.loc[(df['party'] == "R") & (df['last_name'] == name[1]) & (df['first_name'] == name[0])].index
    df.at[rep_index, 'VTITAI'] = 1.0

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,gender,party,...,seniority,ocd_id,state,senate_class,state_rank,lis_id,district,image,VTO20E,VTITAI
10,B001135,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Richard,M.,Burr,None,M,R,...,17,ocd-division/country:us/state:nc,NC,3,senior,S300,NaN,/img/member/b001135_200.jpg,False,1.0


,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,gender,party,...,seniority,ocd_id,state,senate_class,state_rank,lis_id,district,image,VTO20E,VTITAI
113,B001298,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Don,None,Bacon,None,M,R,...,6,ocd-division/country:us/state:ne/cd:2,NE,NaN,NaN,NaN,2,/img/member/115_rp_ne_2_bacon_don_200.jpg,False,0.0


,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,gender,party,...,seniority,ocd_id,state,senate_class,state_rank,lis_id,district,image,VTO20E,VTITAI
166,C001109,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Liz,None,Cheney,None,F,R,...,6,ocd-division/country:us/state:wy/cd:1,WY,NaN,NaN,NaN,At-Large,/img/member/c001109_200.jpg,False,0.0


,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,gender,party,...,seniority,ocd_id,state,senate_class,state_rank,lis_id,district,image,VTO20E,VTITAI
1,B001261,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Barrasso,None,M,R,...,15,ocd-division/country:us/state:wy,WY,1,senior,S317,NaN,/img/member/b001261_200.jpg,False,1.0
3,B001243,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Marsha,None,Blackburn,None,F,R,...,3,ocd-division/country:us/state:tn,TN,1,senior,S396,NaN,/img/member/b001243_200.jpg,False,1.0
5,B000575,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Roy,None,Blunt,None,M,R,...,11,ocd-division/country:us/state:mo,MO,3,senior,S342,NaN,None,False,1.0
7,B001236,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Boozman,None,M,R,...,11,ocd-division/country:us/state:ar,AR,3,senior,S343,NaN,None,False,1.0
8,B001310,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Mike,None,Braun,None,M,R,...,1,ocd-division/country:us/state:in,IN,1,junior,S397,NaN,None,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,W000804,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,J.,Wittman,None,M,R,...,16,ocd-division/country:us/state:va/cd:1,VA,NaN,NaN,NaN,1,/img/member/w000804_200.jpg,False,0.0
537,W000809,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Steve,None,Womack,None,M,R,...,12,ocd-division/country:us/state:ar/cd:3,AR,NaN,NaN,NaN,3,None,False,0.0
538,W000827,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Ron,None,Wright,None,M,R,...,4,ocd-division/country:us/state:tx/cd:6,TX,NaN,NaN,NaN,6,/img/member/w000827_200.jpg,False,0.0
540,Y000033,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Don,None,Young,None,M,R,...,50,ocd-division/country:us/state:ak/cd:1,AK,NaN,NaN,NaN,At-Large,None,False,0.0
